In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder

### Przygotowanie danych

In [2]:
df = pd.read_csv("income_evaluation.csv")
df.columns = df.columns.str.replace(' ', '')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [4]:
#for i, r in df.iteritems():
#    print(pd.unique(df[i]))

In [5]:
df = df.replace(to_replace=' ?', value='Unknown')

In [39]:
x = df.drop(columns=['income'], axis=1).values
y = df['income'].values

In [58]:
scaler = OneHotEncoder()
x_scaler = scaler.fit_transform(x)

In [41]:
x_train, x_test, y_train, y_test = train_test_split(x_scaler, y, test_size=0.2, random_state=42)

### Regresja Logistyczna

In [9]:
random_search = RandomizedSearchCV(LogisticRegression(random_state=7),
                           param_distributions={'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
                           cv=5, n_jobs=-1, verbose=1)

random_search.fit(x_train, y_train)
best = random_search.best_score_
print(best)
print(random_search.best_params_)

C:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   17.1s finished


0.868166461916462
{'solver': 'sag'}


C:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [42]:
reg = LogisticRegression(solver='sag')

In [43]:
reg.fit(x_train, y_train)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [44]:
pred_reg = reg.predict(x_test)

In [45]:
accuracy_score(y_test, pred_reg)

0.8699524028865346

In [46]:
print(y_test)

[' <=50K' ' <=50K' ' >50K' ... ' >50K' ' <=50K' ' <=50K']


In [47]:
print(pred_reg)

[' <=50K' ' >50K' ' >50K' ... ' >50K' ' <=50K' ' <=50K']


In [50]:
precision_score(y_test, pred_reg, pos_label=' >50K')

0.7754946727549468

In [51]:
recall_score(y_test, pred_reg, pos_label=' >50K')

0.648631444939529

### Drzewo decyzyjne

In [52]:
tree = DecisionTreeClassifier()
tree.fit(x_train, y_train)
pred_tree = tree.predict(x_test)

In [53]:
accuracy_score(y_test, pred_tree)

0.8515277138031629

In [55]:
precision_score(y_test, pred_tree, pos_label=' >50K')

0.7144886363636364

In [56]:
recall_score(y_test, pred_tree, pos_label=' >50K')

0.6403564608529599